[Reference](https://neelakash98.medium.com/investing-in-a-portfolio-of-stocks-which-re-balances-every-month-using-pyportfolioopt-f34d9411436d)

In [6]:
pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 6.9 MB/s 
     |████████████████████████████████| 2.8 MB 35.3 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [8]:
pip install cloudcraftz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 19.3 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 228 kB 62.1 MB/s 
     |████████████████████████████████| 930 kB 43.7 MB/s 
     |████████████████████████████████| 6.4 MB 54.6 MB/s 
     |████████████████████████████████| 62 kB 50 kB/s 
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48462 sha256=d5e64c88fea9ea6ede8af6dc23525ff963f18334f3861b8fafd234bf5f2f4f0f
  Stored in directory: /root/.cache/pip/wheels/6a/04/02/2dda6ea59c66d9e685affc8af3a31ad3a5d87b7311689efce6
Successfully built pykalman
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
  

In [1]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from cloudcraftz.utils import financial_summary

import numpy as np 
import yfinance as yf
import pandas as pd
import warnings
import glob
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
# Let's get a set of tickers for our experiment
url = 'https://en.wikipedia.org/wiki/NIFTY_50'
tickers = pd.read_html(url, header=0)[1]
tickers = pd.Series(tickers.Symbol.values).apply(lambda x: x+'.NS')
df = pd.DataFrame(columns=['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'])

for path in tickers:
    data = yf.download(tickers=path, start='2008-01-04', end='2022-06-15')
    data.reset_index(inplace=True)
    
    if data['Date'].loc[0].strftime("%Y-%m-%d")[:4] < '2010':
        data['TIC'] = path.split('/')[-1].replace('.csv', '')
        df = pd.concat([df, data])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
df.reset_index(drop=True, inplace=True)
df.sort_values(['Date', 'TIC'], inplace=True)
df.set_index('Date', inplace=True)
df.head()

,High,Low,Open,Close,Volume,Adj Close,TIC
Date,,,,,,,
2008-01-04,258.959991,246.600006,255.759995,248.070007,2753975,229.801178,ADANIPORTS.NS
2008-01-04,291.500000,281.000000,290.000000,285.500000,139352,262.053589,APOLLOHOSP.NS
2008-01-04,115.790001,111.099998,115.000000,114.089996,208380,99.635605,ASIANPAINT.NS
2008-01-04,214.889999,205.429993,207.000000,210.979996,3574130,188.681641,AXISBANK.NS
2008-01-04,651.174988,642.500000,649.500000,648.862488,226100,449.214661,BAJAJ-AUTO.NS


In [4]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,TIC
Date,,,,,,,
2022-06-14,5498.000000,5324.00,5350.000000,5428.600098,610906,5428.600098,ULTRACEMCO.NS
2022-06-14,731.950012,712.25,721.900024,715.700012,1590761,715.700012,UPL.NS
2022-06-14,453.700012,438.00,440.000000,445.899994,6528192,445.899994,WIPRO.NS
2022-06-15,NaN,NaN,NaN,NaN,NaN,NaN,HINDUNILVR.NS
2022-06-15,NaN,NaN,NaN,NaN,NaN,NaN,TATASTEEL.NS


In [5]:
combined_df = df[['Close', 'TIC']]
combined_df.reset_index(inplace=True)
combined_df.loc[:, 'Date'] = pd.to_datetime(combined_df.loc[:, 'Date'])
combined_df = combined_df.set_index(['Date', 'TIC']).unstack(level=-1)
combined_df.columns = combined_df.columns.droplevel()
combined_df.columns.name = None
combined_df.head()

,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,BRITANNIA.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-04,248.070007,285.500000,114.089996,210.979996,648.862488,2591.0,48.627640,435.377228,89.566666,150.464996,...,111.150002,99.669998,149.788055,887.331299,251.300003,280.000000,76.769997,977.299988,127.833336,111.746254
2008-01-07,245.449997,279.750000,121.239998,218.029999,634.025024,2535.0,47.247997,430.189514,86.733330,152.300003,...,112.180000,97.945000,150.506668,882.043579,243.899994,271.362488,83.544998,988.450012,128.783340,108.551254
2008-01-08,240.960007,273.799988,124.820000,213.149994,634.512512,2537.0,49.555500,448.025146,81.816666,153.285004,...,110.849998,94.139999,148.494553,849.174194,246.962494,275.987488,80.102501,971.500000,125.633331,110.115005
2008-01-09,236.050003,268.950012,120.290001,219.119995,635.037476,2540.0,49.162010,447.221741,77.116669,155.014999,...,109.910004,93.669998,147.593887,851.556030,249.074997,272.262512,83.062500,959.750000,130.050003,110.430008
2008-01-10,224.309998,258.174988,117.224998,217.830002,628.062500,2505.0,47.617195,444.145844,72.866669,155.309998,...,109.599998,88.489998,143.569641,813.780029,245.412506,261.375000,77.790001,925.049988,135.816666,109.732506


In [6]:
combined_df.tail()

,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,BRITANNIA.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-09,735.500000,3670.050049,2686.800049,661.299988,3878.649902,12471.000000,5898.549805,676.349976,332.850006,3347.250000,...,858.849976,754.400024,428.500000,1044.849976,3427.100098,1138.599976,2139.699951,5481.899902,747.299988,475.049988
2022-06-10,726.400024,3700.500000,2708.750000,660.700012,3881.000000,12253.500000,5667.500000,673.349976,331.000000,3360.850098,...,845.700012,756.599976,428.049988,1023.700012,3359.899902,1109.849976,2142.250000,5511.049805,734.400024,460.950012
2022-06-13,695.250000,3651.899902,2660.750000,648.950012,3881.250000,11386.049805,5358.250000,671.299988,317.950012,3359.000000,...,829.500000,735.950012,406.700012,991.950012,3219.600098,1051.949951,2102.399902,5356.149902,726.400024,446.100006
2022-06-14,704.700012,3706.199951,2635.399902,650.750000,3681.750000,11336.500000,5375.750000,682.049988,314.299988,3382.850098,...,825.049988,735.849976,405.149994,996.150024,3210.550049,1030.150024,2102.000000,5428.600098,715.700012,445.899994
2022-06-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
combined_df.isnull().sum()

ADANIPORTS.NS    2
APOLLOHOSP.NS    2
ASIANPAINT.NS    2
AXISBANK.NS      2
BAJAJ-AUTO.NS    2
BAJAJFINSV.NS    2
BAJFINANCE.NS    2
BHARTIARTL.NS    2
BPCL.NS          2
BRITANNIA.NS     2
CIPLA.NS         2
DIVISLAB.NS      2
DRREDDY.NS       2
EICHERMOT.NS     2
GRASIM.NS        2
HCLTECH.NS       1
HDFC.NS          3
HDFCBANK.NS      2
HEROMOTOCO.NS    2
HINDALCO.NS      2
HINDUNILVR.NS    2
ICICIBANK.NS     2
INDUSINDBK.NS    2
INFY.NS          2
ITC.NS           2
JSWSTEEL.NS      2
KOTAKBANK.NS     2
LT.NS            2
M&M.NS           2
MARUTI.NS        2
NESTLEIND.NS     2
NTPC.NS          2
ONGC.NS          2
POWERGRID.NS     2
RELIANCE.NS      2
SBIN.NS          2
SHREECEM.NS      2
SUNPHARMA.NS     2
TATACONSUM.NS    2
TATAMOTORS.NS    2
TATASTEEL.NS     2
TCS.NS           2
TECHM.NS         2
TITAN.NS         2
ULTRACEMCO.NS    2
UPL.NS           2
WIPRO.NS         2
dtype: int64

In [8]:
combined_df.fillna(method='ffill', inplace=True)
combined_df.isnull().sum()

ADANIPORTS.NS    0
APOLLOHOSP.NS    0
ASIANPAINT.NS    0
AXISBANK.NS      0
BAJAJ-AUTO.NS    0
BAJAJFINSV.NS    0
BAJFINANCE.NS    0
BHARTIARTL.NS    0
BPCL.NS          0
BRITANNIA.NS     0
CIPLA.NS         0
DIVISLAB.NS      0
DRREDDY.NS       0
EICHERMOT.NS     0
GRASIM.NS        0
HCLTECH.NS       0
HDFC.NS          0
HDFCBANK.NS      0
HEROMOTOCO.NS    0
HINDALCO.NS      0
HINDUNILVR.NS    0
ICICIBANK.NS     0
INDUSINDBK.NS    0
INFY.NS          0
ITC.NS           0
JSWSTEEL.NS      0
KOTAKBANK.NS     0
LT.NS            0
M&M.NS           0
MARUTI.NS        0
NESTLEIND.NS     0
NTPC.NS          0
ONGC.NS          0
POWERGRID.NS     0
RELIANCE.NS      0
SBIN.NS          0
SHREECEM.NS      0
SUNPHARMA.NS     0
TATACONSUM.NS    0
TATAMOTORS.NS    0
TATASTEEL.NS     0
TCS.NS           0
TECHM.NS         0
TITAN.NS         0
ULTRACEMCO.NS    0
UPL.NS           0
WIPRO.NS         0
dtype: int64

In [9]:
train, test = combined_df[:"2017-06-05"], combined_df["2017-06-05":]

In [10]:
def backtest(cash, backtest_df, weights):
    amount_allocation = {}
    shares, total, balance, total_invested = {}, 0, 0, 0
    prices = backtest_df.iloc[0].to_dict()
    new_prices = backtest_df.iloc[len(backtest_df)-1].to_dict()

    for keys in backtest_df.columns:
        amount_allocation[keys] = weights[keys] * cash

    for keys in backtest_df.columns:
        shares[keys] = (amount_allocation[keys] // prices[keys])

    for keys in backtest_df.columns:
        total_invested = total_invested + (shares[keys] * prices[keys])

    balance = cash - total_invested

    for keys in backtest_df.columns:
        total = total + (shares[keys] * new_prices[keys])

    return total_invested, total, balance
cash = 1000000
initial, getch, balances = [], [], []
dates, port_values = [], []
# Then we start trading.
for i in range(len(test)):
    # Calculate expected returns and sample covariance
    mu = expected_returns.mean_historical_return(train, frequency=22)
    S = risk_models.sample_cov(train, frequency=22)

    # Optimize for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)
    raw_weights = ef.max_sharpe(risk_free_rate=0)
    cleaned_weights = ef.clean_weights()

    # The weights dictionary
    weights = dict(cleaned_weights)

    if (22*i) < len(test):
        new_test = test[(22*i):(22*i+22)+1]
    else:
        break

    invested, total, balance = backtest(cash, new_test, weights)

    initial.append(invested)
    getch.append(total)
    balances.append(balance)
    dates.append(new_test.index[0].strftime("%Y-%m-%d"))

    portfolio = balance + total
    cash = portfolio
    port_values.append(cash)

    train.append(new_test)
fin_df = pd.DataFrame({'date': dates, 'invested': initial, 'fetch': getch, 'balances': balances, 'portfolio': port_values})
fin_df['returns'] = fin_df['portfolio'].pct_change()
fin_df.fillna(0, inplace=True)
financial_summary(fin_df, frequency='M')

,Summary
Start Date,2017-06-05
End Date,2022-05-31
Time Period (in Months),59
Annual Return %,16.26
Annual Volatility,0.181119
Sharpe Ratio,0.897542
Kurtosis,4.829334
Max Drawdown,-0.231777
